In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.listdir("/content/drive/MyDrive")
os.chdir("/content/drive/MyDrive/hmdb51")

Mounted at /content/drive


# Load HMDB51

Before running below code, download HMDB51 first.

In [ ]:
! pip install av
! pip install spikingjelly
#! wget https://raw.githubusercontent.com/pytorch/vision/6de158c473b83cf43344a0651d7c01128c7850e6/references/video_classification/transforms.py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.6/437.6 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import random_split, DataLoader
from torch.optim.lr_scheduler import StepLR
import torchvision
from torchvision import get_video_backend
from torchvision.models.video import r3d_18
from torchvision import transforms
import os
from tqdm.auto import tqdm
import numpy as np
import time
import datetime
import random
import transforms as T
import av

from spikingjelly.activation_based import layer, neuron, surrogate, encoding, functional

In [ ]:
# Datasets and Dataloaders for model training ..

val_split = 0.05 # 검증 데이터 비율 (5%)
num_frames = 20
clip_steps = 50 # 연속된 프레임 사이의 간격
num_workers = 8
pin_memory = True # GPU 사용 시 데이터 로딩 성능 최적화
train_tfms = torchvision.transforms.Compose([
                                 T.ToFloatTensorInZeroOne(), # 0~1 범위의 float32 tensor
                                 T.Resize((128, 171)), # 128x171
                                 T.RandomHorizontalFlip(),# 랜덤으로 좌우 반전
                                 T.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989]),
                                 T.RandomCrop((112, 112)) # 112x112
                               ])
test_tfms =  torchvision.transforms.Compose([
                                             T.ToFloatTensorInZeroOne(),
                                             T.Resize((128, 171)),
                                             T.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989]),
                                             T.CenterCrop((112, 112))
                                             ])
hmdb51_train = torchvision.datasets.HMDB51('video_data/', 'test_train_splits/', num_frames,
                                                step_between_clips = clip_steps, fold=1, train=True,
                                                transform=train_tfms, num_workers=num_workers)


hmdb51_test = torchvision.datasets.HMDB51('video_data/', 'test_train_splits/', num_frames,
                                                step_between_clips = clip_steps, fold=1, train=False,
                                                transform=test_tfms, num_workers=num_workers)

total_train_samples = len(hmdb51_train)
total_val_samples = round(val_split * total_train_samples)

print(f"number of train samples {total_train_samples}")
print(f"number of validation samples {total_val_samples}")
print(f"number of test samples {len(hmdb51_test)}")

  0%|          | 0/417 [00:14<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
batch_size = 32
num_workers = 0

kwargs = {'num_workers':num_workers, 'pin_memory':True} if torch.cuda.is_available() else {'num_workers':num_workers}
#kwargs = {'num_workers':num_workers}
#kwargs = {}

hmdb51_train_v1, hmdb51_val_v1 = random_split(hmdb51_train, [total_train_samples - total_val_samples,
                                                                       total_val_samples])

#hmdb51_train_v1.video_clips.compute_clips(16, 1, frame_rate=30)
#hmdb51_val_v1.video_clips.compute_clips(16, 1, frame_rate=30)
#hmdb51_test.video_clips.compute_clips(16, 1, frame_rate=30)

#train_sampler = RandomClipSampler(hmdb51_train_v1.video_clips, 5)
#test_sampler = UniformClipSampler(hmdb51_test.video_clips, 5)

train_loader = DataLoader(hmdb51_train_v1, batch_size=batch_size, shuffle=True, **kwargs)
val_loader   = DataLoader(hmdb51_val_v1, batch_size=batch_size, shuffle=True, **kwargs)
test_loader  = DataLoader(hmdb51_test, batch_size=batch_size, shuffle=False, **kwargs)

In [ ]:
batch = next(iter(train_loader))
print(f"Batch type: {type(batch)}")
print(f"Batch length: {len(batch)}")

video, audio, label = next(iter(train_loader))
print(video.shape) # (batch size, channels, frames, height, width)
print(audio.shape)
print(label.shape) # (batch size)

Batch type: <class 'list'>
Batch length: 3
torch.Size([16, 3, 16, 112, 112])
torch.Size([16, 1, 0])
torch.Size([16])


In [ ]:
import joblib

# 데이터 로더 저장
joblib.dump(train_loader, "hmdb51_train.pkl")
joblib.dump(test_loader, "hmdb51_test.pkl")

앞으로는 이것만 하면 됨

In [ ]:
import joblib

# 저장된 데이터 로더 불러오기
train_loader = joblib.load("hmdb51_train.pkl")
test_loader = joblib.load("hmdb51_test.pkl")

/usr/local/lib/python3.11/dist-packages/torchvision/datasets/video_utils.py:219: UserWarning: There aren't enough frames in the current video to get a clip for the given clip length and frames between clips. The video (and potentially others) will be skipped.
  warnings.warn(


In [ ]:
print(f'Number of batches in train_loader: {len(train_loader)}')
print(f'num_workers: {train_loader.num_workers}')
print(f'Batch size: {train_loader.batch_size}')

video, audio, label = next(iter(train_loader))
print(f"Video shape: {video.shape}")  # (batch_size, frames, height, width)
print(f"Audio shape: {audio.shape}")  # 오디오 데이터 크기
print(f"Label shape: {label.shape}")  # (batch_size,)

Number of batches in train_loader: 221
num_workers: 0
Batch size: 32


/usr/local/lib/python3.11/dist-packages/torchvision/io/video.py:169: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


AttributeError: module 'av' has no attribute 'AVError'

# ANN

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import time
import matplotlib.pyplot as plt

In [ ]:
class ANN(nn.Module):
    def __init__(self, input_size, output_size):
        super(ANN, self).__init__()
        self.network = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_size, output_size),
            nn.ReLU() # 출력값은 항상 0 이상, 출력 크기는 (batch_size, output_size)=(16,51)
        )

    def forward(self, x):
        x = self.network(x)
        return x

'''
class SNN(nn.Module):
    def __init__(self, input_size, output_size, tau=10.0, v_th=1.0, dt=1.0):
        super(SNN, self).__init__()
        self.weights = None
        self.lif = neuron.LIFNode(tau=tau, v_threshold=v_th, detach_reset=True)

    def set_weights(self, weights, photo_responsivity):
        self.weights = weights * (max(abs(photo_responsivity)) / max(abs(weights)))

    def forward(self, x):
        if self.weights is None:
          raise valueError("Weights have not been set. Use set_weights() first.")

        membrane_potential = torch.matmul(input_spikes, self.weights)
        return self.lif(membrane_potential)
'''

'\nclass SNN(nn.Module):\n    def __init__(self, input_size, output_size, tau=10.0, v_th=1.0, dt=1.0):\n        super(SNN, self).__init__()\n        self.weights = None\n        self.lif = neuron.LIFNode(tau=tau, v_threshold=v_th, detach_reset=True)\n\n    def set_weights(self, weights, photo_responsivity):\n        self.weights = weights * (max(abs(photo_responsivity)) / max(abs(weights)))\n\n    def forward(self, x):\n        if self.weights is None:\n          raise valueError("Weights have not been set. Use set_weights() first.")\n\n        membrane_potential = torch.matmul(input_spikes, self.weights)\n        return self.lif(membrane_potential)\n'

**Train**

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
input_size = 3 * 20 * 112 * 112
output_size = 51
ann = ANN(input_size, output_size).to(device)

epochs = 100
learning_rate = 0.01
criterion = nn.MSELoss()
optimizer = optim.SGD(ann.parameters(), lr=learning_rate)

In [ ]:
checkpoint_path = "/content/drive/MyDrive/commit/graduate/hmdb51ann_checkpoint.pth"
start_epoch = 0
max_test_acc = -1
train_acc_list = []
test_acc_list = []

# check point 불러오기
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    ann.load_state_dict(checkpoint["model_state"])
    optimizer.load_state_dict(checkpoint["optimizer_state"])
    start_epoch = checkpoint["epoch"]
    max_test_acc = checkpoint["max_test_acc"]
    train_acc_list = checkpoint["train_acc_list"]
    test_acc_list = checkpoint["test_acc_list"]
    print(f"체크포인트 불러옴: epoch {start_epoch}, max_test_acc={max_test_acc*100:.2f}%")
else:
    print("새로운 학습을 시작합니다.")


for epoch in range(start_epoch, epochs):
  start_time = time.time()
  ann.train()
  train_loss = 0
  train_acc = 0
  train_samples = 0
  for frame, _, label in train_loader:
    optimizer.zero_grad()
    frame = frame.to(device)
    #frame = frame.to(device).permute(2,0,1,3,4) # [N,T,C,H,W] -> [T,N,C,H,W]
    label = label.to(device)
    label_onehot = F.one_hot(label,  51).float()

    out_fr = ann(frame) # 크기: (batch_size, output_size)=(32,51)
    loss = F.mse_loss(out_fr, label_onehot)
    loss.backward()
    optimizer.step()

    train_samples += label.numel()
    train_loss += loss.item() * label.numel()
    train_acc += (out_fr.argmax(1) == label).float().sum().item()
  train_time = time.time()
  train_speed = train_samples / (train_time - start_time)
  train_loss /= train_samples
  train_acc /= train_samples
  train_acc_list.append(train_acc)

  print(f'epoch {epoch}: train_loss={train_loss*100:.2f}%, train_acc={train_acc*100:.2f}%, train_speed={train_speed:.4f}images/s')

  ann.eval()
  test_loss = 0
  test_acc = 0
  test_samples = 0
  with torch.no_grad():
    for frame, _, label in test_loader:
      frame = frame.to(device)
      #frame = frame.permute(2,0,1,3,4)  # [N, T, C, H, W] -> [T, N, C, H, W]
      label = label.to(device)
      label_onehot = F.one_hot(label, 51).float()
      out_fr = ann(frame)
      loss = F.mse_loss(out_fr, label_onehot)
      test_samples += label.numel()
      test_loss += loss.item() * label.numel()
      test_acc += (out_fr.argmax(1) == label).float().sum().item()
  test_time = time.time()
  test_speed = test_samples / (test_time - train_time)
  test_loss /= test_samples
  test_acc /= test_samples
  test_acc_list.append(test_acc)

  if test_acc > max_test_acc:
    max_test_acc = test_acc

  print(f'epoch = {epoch}, test_loss={test_loss*100:.2f}%, test_acc={test_acc*100:.2f}%, max_test_acc={max_test_acc*100:.2f}%, test_speed={test_speed:.4f}images/s')
  print(f'escape time = {(datetime.datetime.now() + datetime.timedelta(seconds=(time.time() - start_time) * (epochs - epoch))).strftime("%Y-%m-%d %H:%M:%S")}\n')

  checkpoint = {
        "epoch": epoch + 1,  # 다음 epoch부터 학습을 이어가므로 1 증가
        "model_state": ann.cpu().state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "max_test_acc": max_test_acc,
        "train_acc_list": train_acc_list,
        "test_acc_list": test_acc_list
  }
  torch.save(checkpoint, checkpoint_path)
  ann.to(device)


# weight 저장
# torch.save(ann.state_dict(), "ann_weights.pth")

새로운 학습을 시작합니다.
epoch 0: train_loss= nan, train_acc= 0.0448, train_speed= 2.2333
epoch 1: train_loss= nan, train_acc= 0.0520, train_speed= 22.3591
epoch 2: train_loss= nan, train_acc= 0.0520, train_speed= 22.2115
epoch 3: train_loss= nan, train_acc= 0.0520, train_speed= 21.6722
epoch 4: train_loss= nan, train_acc= 0.0520, train_speed= 22.2979
epoch 5: train_loss= nan, train_acc= 0.0520, train_speed= 22.3251
epoch 6: train_loss= nan, train_acc= 0.0520, train_speed= 22.2702
epoch 7: train_loss= nan, train_acc= 0.0520, train_speed= 22.2923
epoch 8: train_loss= nan, train_acc= 0.0520, train_speed= 22.2352
epoch 9: train_loss= nan, train_acc= 0.0520, train_speed= 20.5849
epoch 10: train_loss= nan, train_acc= 0.0520, train_speed= 20.6500
epoch 11: train_loss= nan, train_acc= 0.0520, train_speed= 20.5828
epoch 12: train_loss= nan, train_acc= 0.0520, train_speed= 20.6199
epoch 13: train_loss= nan, train_acc= 0.0520, train_speed= 20.6853
epoch 14: train_loss= nan, train_acc= 0.0520, train_speed=

# SNN

In [ ]:
class SNN(nn.Module):
  def __init__(self, input_size, output_size, tau=2.0):
    super(SNN, self).__init__()
    self.network = nn.Sequential(
      nn.Flatten(),
      nn.Linear(input_size, output_size),
      neuron.LIFNode(tau=tau)
    )

  def forward(self, x):
    x = self.network(x)
    return x

In [ ]:
# weight 불러오기
ann_weights = torch.load("ann_weights.pth", map_location=device)

# snn weight으로 변환
max_weight = max(abs(w.max().item()) for w in ann_weights.value())
snn_weights = {k: v / max_weight for k, v in ann_weights.items()}

# SNN 초기화 및 snn_weights 적용
snn = SNN(input_size, output_size).to(device)
snn.load_state_dict(snn_weights)

In [ ]:
# train
snn.eval()
test_loss = 0
test_acc = 0
test_samples = 0

with torch.no_grad():
  for frame, _, label in test_loader:
    frame = frame.to(device)
    frame = frame.permute(2,0,1,3,4)  # [N, T, C, H, W] -> [T, N, C, H, W]
    label = label.to(device)
    label_onehot = F.one_hot(label, 51).float()
    out_fr = snn(frame)
    loss = F.mse_loss(out_fr, label_onehot)
    test_samples += label.numel()
    test_loss += loss.item() * label.numel()
    test_acc += (out_fr.argmax(1) == label).float().sum().item()
test_time = time.time()
test_speed = test_samples / (test_time - train_time)
test_loss /= test_samples
test_acc /= test_samples
test_acc_list.append(test_acc)